<a href="https://colab.research.google.com/github/chetankhairnar05/Python_Automation/blob/main/tables_from_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Task: you have pdf and write code to extract tables from pdf and save in xl files


#libraries : tabula or camelot(works well)


# First, you must install the required Python library.
# You also need to have Java installed on your system for tabula-py to work, as it's a wrapper for the Tabula-Java library.
# You can run this command in your terminal or a Colab cell:
!pip install tabula-py pandas
!pip install xlsxwriter

import pandas as pd
import tabula

# This URL points to a public PDF document with tables that are similar to the one you provided.
# It has a clear grid structure and is a good test case for the 'lattice' option.
pdf_url = "https://assets.accessible-digital-documents.com/uploads/2017/01/sample-tables.pdf"
output_filename = "extracted_tables.xlsx"

# --- Step 1: Read the tables from the PDF using a more robust method ---
# The 'lattice=True' parameter is crucial for tables with visible lines and merged cells.
# It uses the grid lines to define the cells, which is more reliable than the 'stream' method's whitespace-based parsing.
# You can also use the 'area' parameter to specify the coordinates of the table,
# for example: area=[top, left, bottom, right] to get even more precise results.
print(f"Extracting tables from PDF: {pdf_url}")
try:
    # Use lattice=True for tables with a clear grid structure.
    dfs = tabula.read_pdf(pdf_url, pages="all", multiple_tables=True, lattice=True)
    print(f"Successfully extracted {len(dfs)} tables.")
except Exception as e:
    print(f"Error extracting tables from the PDF: {e}")
    # An empty list is fine if no tables are found, but a more specific error is useful
    dfs = []

# --- Step 2: Process and Display the extracted DataFrames ---
if len(dfs) > 0:
    for i, df in enumerate(dfs):
        print(f"\n--- Table {i + 1} from the PDF ---")
        print(df.head()) # Print the first 5 rows of each extracted table

# --- Step 3: (Optional) Export all tables to a single Excel file on separate sheets ---
if len(dfs) > 0:
    try:
        with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
            for i, df in enumerate(dfs):
                sheet_name = f"Table_{i + 1}"
                # Handle potential MultiIndex columns
                if isinstance(df.columns, pd.MultiIndex):
                    # Flatten the MultiIndex columns into a single string for Excel
                    df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else str(col) for col in df.columns.values]
                    df.columns = [col.replace('nan_', '').replace('_nan', '') for col in df.columns]

                df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"\nSuccessfully exported all tables to '{output_filename}'.")
    except Exception as e:
        print(f"\nError exporting tables to Excel: {e}")

# --- Step 4: (Optional) Download the Excel file in a Colab environment ---
# This part is only for use in Google Colab. If you're running locally, you don't need this.
try:
    from google.colab import files
    print("Downloading the Excel file to your local machine.")
    files.download(output_filename)
except ImportError:
    print("\nNote: 'google.colab' module not found. This download step is skipped.")


Extracting tables from PDF: https://assets.accessible-digital-documents.com/uploads/2017/01/sample-tables.pdf


Aug 07, 2025 10:52:36 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Aug 07, 2025 10:52:36 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Aug 07, 2025 10:52:36 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Aug 07, 2025 10:52:36 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 07, 2025 10:52:36 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 07, 2025 10:52:37 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 07, 2025 10:52:37 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



Successfully extracted 29 tables.

--- Table 1 from the PDF ---
  Column header (TH) Column header (TH).1 Column header (TH).2
0    Row header (TH)       Data cell (TD)       Data cell (TD)
1     Row header(TH)       Data cell (TD)       Data cell (TD)

--- Table 2 from the PDF ---
  Expenditure by function £ million            2009/10  2010/11 1  Unnamed: 0
0                  Policy functions          Financial      22.50       30.57
1                               NaN      Information 2      10.20       14.80
2                               NaN        Contingency       2.60        1.20
3             Remunerated functions  Agency services 3      44.70       35.91
4                          Payments              22.41      19.88         NaN

--- Table 3 from the PDF ---
  Main character Daniel Radcliffe
0     Sidekick 1     Rupert Grint
1     Sidekick 2      Emma Watson
2   Lovable ogre  Robbie Coltrane
3      Professor     Maggie Smith
4     Headmaster   Richard Harris

--- Table 4 fr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# First, you must install the required Python libraries and their dependencies.
# You will need to install ghostscript, which is a dependency for camelot.
# On Linux (e.g., in a Colab environment), you can install it like this:
# !apt-get install ghostscript
# On Windows, you need to install it separately from the official Ghostscript website.
#
# Then, install the Python libraries:
!pip install camelot-py[cv] pandas
!pip install xlsxwriter

import pandas as pd
import camelot

# This URL points to a public PDF document with tables.
pdf_url = "https://assets.accessible-digital-documents.com/uploads/2017/01/sample-tables.pdf"
output_filename = "extracted_tables.xlsx"

# --- Step 1: Read the tables from the PDF using camelot ---
# We'll use the 'lattice' flavor since the table in your example has clear lines.
# The 'pages="all"' parameter tells camelot to process all pages in the PDF.
print(f"Extracting tables from PDF using Camelot: {pdf_url}")
try:
    # Use flavor='lattice' for tables with a clear grid structure.
    tables = camelot.read_pdf(pdf_url, pages="all", flavor='lattice')
    print(f"Successfully extracted {tables.n} tables.")
except Exception as e:
    print(f"Error extracting tables from the PDF: {e}")
    tables = []

# --- Step 2: Process and Display the extracted DataFrames ---
if tables:
    for i, table in enumerate(tables):
        df = table.df # Get the DataFrame from the Camelot table object
        print(f"\n--- Table {i + 1} from the PDF ---")
        print(df.head()) # Print the first 5 rows of each extracted table

# --- Step 3: (Optional) Export all tables to a single Excel file on separate sheets ---
if tables:
    try:
        with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
            for i, table in enumerate(tables):
                df = table.df # Get the DataFrame from the Camelot table object
                sheet_name = f"Table_{i + 1}"
                # Handle potential MultiIndex columns
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else str(col) for col in df.columns.values]
                    df.columns = [col.replace('nan_', '').replace('_nan', '') for col in df.columns]

                df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"\nSuccessfully exported all tables to '{output_filename}'.")
    except Exception as e:
        print(f"\nError exporting tables to Excel: {e}")

# --- Step 4: (Optional) Download the Excel file in a Colab environment ---
# This part is only for use in Google Colab. If you're running locally, you don't need this.
try:
    from google.colab import files
    print("Downloading the Excel file to your local machine.")
    files.download(output_filename)
except ImportError:
    print("\nNote: 'google.colab' module not found. This download step is skipped.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.9 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


Extracting tables from PDF using Camelot: https://assets.accessible-digital-documents.com/uploads/2017/01/sample-tables.pdf
Successfully extracted 28 tables.

--- Table 1 from the PDF ---
                                   0                  1        2          3
0  Expenditure by function £ million                     2009/10  2010/11 1
1                   Policy functions          Financial     22.5      30.57
2                                         Information 2     10.2       14.8
3                                           Contingency      2.6        1.2
4              Remunerated functions  Agency services 3     44.7      35.91

--- Table 2 from the PDF ---
                                  0                        1
0  Main character  Daniel Radcliffe                         
1                        Sidekick 1             Rupert Grint
2                        Sidekick 2              Emma Watson
3                      Lovable ogre          Robbie Coltrane
4                    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>